<h2>Twitter Movie Success: Getting Data (Draft 2)</h2>
Thomas Le

In [ ]:
import tweepy
import pandas as pd 
import csv
import GetOldTweets3 as got
import numpy as np
import re

import itertools
import collections
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from textblob import TextBlob

import time
import datetime
from datetime import timedelta

In [ ]:
consumer_key = 'm3EuCXkTyC830AHQSCDzAQ6SD' 
consumer_secret = 'GjagyDgwlk8mjRoVncXaS1bAjkKv92hBuPzpTuReoX2uBHZ2MW'

access_token = '1158877831113465856-n2zYtW8kKViUJ3mYA3Vt86VmZcm536'
access_token_secret = 'UDbd3Xaqaf6zjtnzvQ5mk2BEuB2PcWsiZRyHNzk707bYQ'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
#for importing files

bo_data = pd.read_csv('box_office_data.csv') 
display(bo_data)

<h2>Functions</h2>

In [ ]:
# this function takes all the words in the tweets given, splits them and removes all stop words, title name, symbols, and popular occuring movie related words. The remaining words, the top 10 are taken and analysed for their sentiment value (-1 to 1). The sentiment value of these top 10 words are taken and averaged to give single aggregate sentiment value for that query.

def averageSentiment(movie_title, all_tweets):
    
    topwords_list = []
    
    collection_words = movie_title.lower().split()
    symbols = ['!','|',',','!','...','-']
    random = ['movie','movies','@youtube','trailer','video','official']
    stop_words = set(stopwords.words('english') + collection_words + symbols + random)
    
    words_in_tweets = []
    for tweets in all_tweets:
        words_in_tweets.append(tweets.lower().split())
    
    tweets_nsw = []

    for tweet_words in words_in_tweets:
        for word in tweet_words:
            if not word in stop_words:
                tweets_nsw.append(word) #nsw: no stop words
                
    all_words= list(itertools.chain(tweets_nsw))
    counts= collections.Counter(all_words)
    topwords_list = np.array(counts.most_common(10)) 
    topwords_list = topwords_list[:,0]
    
    sentiments = [TextBlob(tweet) for tweet in topwords_list]
    polarities = [word.sentiment.polarity for word in sentiments]  

    avgsentiment = sum(polarities)/len(polarities)
    avgsentiment = round(avgsentiment,4)
    
    return(avgsentiment)

In [ ]:
# function scrapes Twitter for tweets for a query between certain dates and returns a list of: the movie title, number of tweets, number of likes for all those tweets, number of retweets for all those tweets, and the average sentiment.

#global variables
listno = []
tweet_data = []
list_of_tweets = []
retweet_list = []
output = []
query= 0

def scraper(listno, query, since, until, maxtweets, output):
    
    #put them here again to reset everytime a movie is searched
    listno = []
    list_of_tweets = []
    retweet_list = []
    
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                                           .setSince(since)\
                                           .setUntil(until)\
                                           .setMaxTweets(maxtweets)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    for tweet in tweets:
        listno.append(tweet.favorites)
        list_of_tweets.append(tweet.text)
        retweet_list.append(tweet.retweets)
    
    avgsentiment = averageSentiment(query, list_of_tweets)
    
    listno2 = [query] +[len(listno)]+[sum(listno)]+[sum(retweet_list)]+[avgsentiment]
    
    output.append(listno2)
    
    return output

In [ ]:
#second scraper only difference is: adds query to hashtag line

#global variables
listno = []
tweet_data2 = []
list_of_tweets = []
retweet_list = []
output = []

def scraper2(listno, query, since, until, maxtweets, output):
    
    #put them here again to reset everytime a movie is searched
    listno = []
    list_of_tweets = []
    retweet_list = []
    
    query = str('#') + ''.join(query.lower().split())
    
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                                           .setSince(since)\
                                           .setUntil(until)\
                                           .setMaxTweets(maxtweets)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    for tweet in tweets:
        listno.append(tweet.favorites)
        list_of_tweets.append(tweet.text)
        retweet_list.append(tweet.retweets)
    
    avgsentiment = averageSentiment(query, list_of_tweets)
    
    listno2 = [query] +[len(listno)]+[sum(listno)]+[sum(retweet_list)]+[avgsentiment]
    
    output.append(listno2)
    
    return output

<h2>Twitter: Do more people use hashtags or text?</h2>
The following two code cell blocks are here to demonstrate some insights into user behaviour on Twitter. Disclaimer: The sample for these was very small (limited processing power to scrape more tweets), so more research and data is warranted before taking any actions.
<p></p>
Four movies were tested by querying both their title as text in addition to their title as a hashtag. 
<p></p>
The way the GetOldTweets3 library works in this context is that a query of 'Zootopia' will return tweets showing both 'Zootopia' <b>and</b> '#zootopia', so for the number of tweets shown for 'Zootopia' that will include the total count for tweets with and without the hashtag.
<p></p>
For movie titles with names that are multiple words, the query of 'Finding Dory' for example will only return those with 'Finding Dory' (including the space). For the total counts for these, adding the number of tweets for both 'Finding Dory' and '#findingdory' will be necessary to get the total number of tweets about that movie.
<p></p>
Though testing this on movies with a single word title may seem redundant, I thought it prudent for the sake of seeing if having a shorter title had any affect on people more easily hashtagging it.

In [ ]:
# script to test whether hashtags or regular name is a better search query
# sqt = search query table

#this part to create the table
sqt_data = pd.read_csv('box_office_data.csv', index_col = 'Movie Title') 
sqt = sqt_data.loc[['Finding Dory', 'Suicide Squad', 'Zootopia', 'Deadpool']]
sqt = sqt.reset_index()[['Movie Title','Date Released']]
#display(sqt)

hashtag_list = []

for i in sqt['Movie Title']:
    tagged = str('#') + ''.join(i.lower().split())
    hashtag_list.append(tagged)
   
sqt2 = pd.DataFrame(hashtag_list, columns = ['Movie Title'])
sqt2['Date Released'] = sqt['Date Released']
sqt = sqt.append(sqt2, sort = False) #ignore_index = true
sqt.sort_values(by=['Date Released'], inplace=True, ascending=True)
display(sqt)
print('')

In [ ]:
print('STARTED SCRAPING SQT TABLE AT')
print(datetime.datetime.now())

In [ ]:
#ok now actually seeing which has more tweets

list_sqt = []

sinceDELTA = timedelta(weeks=5)
untilDELTA = timedelta(weeks=4, days=6)
sqt_output = []

for i in sqt['Movie Title']:
   movie_title = i
   for j in sqt['Date Released']:
        j = datetime.datetime.strptime(j, '%m/%d/%y').date()
        since = str(j - sinceDELTA)
        until = str(j - untilDELTA)
   scraper(list_sqt, movie_title, since, until, 0, sqt_output)

sqt_test = pd.DataFrame(sqt_output, columns = ['movie title','total number of tweets','total likes','retweets','average tweet sentiment'])
display(sqt_test)
sqt_test.to_csv('hashtag_or_regular_text.csv')

So it seems that the average user of Twitter does not use hashtags as much as the text equivalent of a movie title. From these samples, we see that about 15% of all tweets (with some form of the movie title) are posted with the hashtag version as opposed to 85% using the movie title as just text.
<p></p>
Initial hypothesis from this finding was that the average Twitter user uses twitter more to let their friends know what they're up to, whereas users using hashtags are trying to promote a movie.
<p></p>
This hypothesis is supported by the higher percentage of likes and retweets for the tweets queried with hashtags. Perhaps users with more followers (movie accounts, actors, influencers, etc) are tweeting with hashtags and garnering more likes and retweets, but the total users talking about the movie casually to their twitter social circles use just the text form of the movie title.
<p></p>
These findings are not suggesting any action, only thought useful to keep in mind during any subsequent data scraping on twitter and for the remainder of this project.

<h2>Gathering final set of movie data</h2>
If the movie has a one word title, then only query by its title and the returned tweets analysis will include tweets for both the text version 'MovieTitle' and the tweets with a hashtag version '#movietitle'.
<p></p>
If the movie has a multiple word title, then run two queries: one query to return the tweets of the text version 'Multiple Word Movie Title' and one query to return the tweet analysis for the hashtag version '#multiplewordmovietitle'. These analyses will be added together to return one final row of data for the tweet analysis of that movie.

In [ ]:
print('STARTED SCRAPING TABLE AT')
print(datetime.datetime.now())

In [ ]:
sinceDELTA = timedelta(weeks=5)
untilDELTA = timedelta(weeks=4, days=6)
maxtweets = 0
tweet_data = []
date_counter = 0

for i in bo_data['Movie Title']:
    titleLength = len(i.split())
    
    movie_title = i
    movie_hashtag = ''.join(movie_title.lower().split()) #weird zootopia glitch happens unless i do this (theres already a hashtag form statement in scraper2
    
    #increment date so the time period (of which to scrape for tweets) will change for each movie depending on day of release
    j = datetime.datetime.strptime(bo_data['Date Released'][date_counter], '%m/%d/%y').date()
    since = str(j - sinceDELTA)
    until = str(j - untilDELTA) 
    date_counter += 1
    
    if titleLength > 1:
        
        multiple_list = []
        multiple_list_h = [] #hashtagged
        tweets1 = []
        tweets2 = []

        #regular text
        a = scraper(multiple_list, movie_title, since, until, maxtweets, tweets1)
        a = np.array(a[0])

        #hashtag text
        b = scraper2(multiple_list_h, movie_hashtag, since, until, maxtweets, tweets2)
        b = np.array(b[0])
        
        #empty out previous loop of combined regular text and hashtag text values    
        list3 = []
        
        #combining the tweet values for both regular and hashtag
        for k in range (1,5):
            c = float(a[k])+ float(b[k]) #loops through each tweet analysis value
            list3.append(c)   
        list3.insert(0, movie_title) #insert original movie title into row of data
        
        #append combined data (row of 1 movie data) into list of rows
        tweet_data.append(list3)

    elif titleLength == 1:
        
        single_list = [] 
        tweets3 = []
            
        #both regular text and hashtag text for single word movie title
        d = scraper(single_list, movie_title, since, until, 0, tweets3)
        d = np.array(d[0])
        
        #append row of 1 movie data into list of rows
        tweet_data.append(d)
  
twitter_movie_data = pd.DataFrame(tweet_data, columns = ['movie title','total number of tweets','total likes','retweets','average tweet sentiment'])
display(twitter_movie_data)

twitter_movie_data.to_csv('twitter_movie_data.csv')

In [ ]:
print('COMPLETED SCRAPING TABLE AT')
print(datetime.datetime.now())

In [ ]:
twitter_movie_data = pd.DataFrame(tweet_data, columns = ['movie title','total number of tweets','total likes','retweets','average tweet sentiment'])

display(twitter_movie_data)

twitter_movie_data.to_csv('twitter_movie_data.csv', index = False)